# [Программирование на Python (SCS)](https://compscicenter.ru/courses/python/2015-autumn/classes/)

## Лектор Сергей Лебедев:  sergei.a.lebedev@gmail.com


|     **Дата**     |   **Название**  |     |
|:----------------:|:---------------:|:-----------------:|
| 12 октября 2015      |    Классы 1 | 

# 6. Классы часть 1
### Синтаксис объявления классо

In [2]:
class Counter:
    """I count. That is all."""
    def __init__(self, initial=0): # конструктор
        self.value = initial       # запись атрибута

    def increment(self):
        self.value += 1

    def get(self):
        return self.value          # чтение атрибута

c = Counter(42)
c.increment()
c.get()

43

В отличие от Java и C++ в Python нет “магического” ключевого слова this. Первый аргумент конструктора
__init__ и всех остальных методов — экземпляр класса, который принято называть self.
Синтаксис языка не запрещает называть его по-другому, но так делать не рекомендуется.
- Аналогично другим ООП языкам Python разделяет атрибуты экземпляра и атрибуты класса.
- Атрибуты добавляются к экземпляру посредством присваивания к <font color=green>self</font> конструкцией вида:
```
self.some_attribute = value
```
- Атрибуты класса объявляются в теле класса или прямым присваиванием к классу:

In [15]:
class Counter:
    all_counters = []

    def __init__(self, initial=0):
        Counter.all_counters.append(self)
                
Counter.some_other_attribute = 42


In [4]:
# Изменяемый тип (лист) разделяется ВСЕМИ объектами!!!!!
Counter.all_counters = [1, 2]
c = Counter()
d = Counter()

print(Counter.all_counters)
print(c.all_counters)
print(d.all_counters)  

[1, 2]
[1, 2]
[1, 2]


# Слайд 4. Соглашения об именовании атрибутов и методов

В Python нет модификаторов доступа к атрибутам и методам: почти всё можно читать и присваивать.

Для того чтобы различать публичные и внутренние атрибуты визуально, к внутренним атрибутам добавляют в начало символ подчеркивания:
```
class Noop:
    some_attribute = 42
    _internal_attribute = []
```

Особо ярые любители контроля используют два подчёркивания. Доступ к такому элементу доступен только через `название класса + двоёное подчёркивание + имя аттрибута`:


In [5]:
class Noop:
    some_attribute = 42
    __very_internal_attribute = []
Noop._Noop__very_internal_attribute

[]

# Слайд 5. Привет от калибровочного теста


In [1]:
from collections import deque


class MemorizingDict(dict):
    """Особый словарь, который сохраняет 10 последних ключей
        
    собственно, наследуемся от словаря, init по умолчанию (словарь)
    имеем аттрибут класса "history". инит в хистори не пишет, только
    метод сет.
"""
    history = deque(maxlen=10)
    def set(self, key, value):
        self.history.append(key)
        self[key] = value
    def get_history(self):
        return self.history

d = MemorizingDict({"foo": 42})
print(d)
d.set("baz", 100500)
print(d.get_history()) # ==> ?

{'foo': 42}
deque(['baz'], maxlen=10)


In [2]:
# При создании нового объекта нашего особого словаря, изменяемый
# объект history уже существует и содержит данные!
d = MemorizingDict()
d.set("boo", 500100)
print(d.get_history()) # ==> deque(['baz', 'boo'], maxlen=10)

deque(['baz', 'boo'], maxlen=10)


# Слайд 6. Внутренние атрибуты классов и экземпляров


In [29]:
class Noop:
    """I do nothing at all."""
Noop.__doc__

'I do nothing at all.'

In [23]:
Noop.__name__

'Noop'

In [25]:
Noop.__module__

'__main__'

In [26]:
Noop.__bases__

(object,)

In [30]:
noop = Noop()
noop.__class__

__main__.Noop

In [32]:
noop.__dict__ # словарь атрибутов объекта

{}

In [34]:
noop.some_new_attribute = 42
noop.__dict__

{'some_new_attribute': 42}

# Слайд 7. Подробнее о __dict__

Добавление, изменение и удаление атрибутов — это фактически операции со словарём.

In [1]:
noop.__dict__["some_other_attribute"] = 100500
noop.some_other_attribute
del noop.some_other_attribute

NameError: name 'noop' is not defined

Поиск значения атрибута происходит динамически в
момент выполнения программы.

Для доступа к словарю атрибутов можно также использовать функцию <font color=green>vars</font>:

In [35]:
vars(noop)  # не используем "закрытый" аттрибут __dict__

{'some_new_attribute': 42}

In [36]:
vars(Noop)  # vars работает и с самим классом

mappingproxy({'__module__': '__main__',
              '__doc__': 'I do nothing at all.',
              '__dict__': <attribute '__dict__' of 'Noop' objects>,
              '__weakref__': <attribute '__weakref__' of 'Noop' objects>})


# Слайд 8. __slots__

С помощью специального аттрибута класса `__slots__`
можно зафиксировать множество возможных атрибутов
экземпляра (как, например, для класса Point(x, y))

Экземпляры класса с указанным `__slots__` требуют
меньше памяти, потому что у них отсутствует `__dict__`.

P.S. По сути slots не нужен, т.к. есть **namedtuple**!!! Но, если нужно добавить методы, то можно использовать **namedtuple** в качестве базового класса

In [2]:
class Noop:
    __slots__ = ["some_attribute"]
noop = Noop()
noop.some_attribute = 42
noop.some_attribute

42

In [3]:
noop.some_other_attribute = 100500  # вызовет ошибку, т.к. аттрибуты добавлять теперь нельзя

AttributeError: 'Noop' object has no attribute 'some_other_attribute'

In [4]:
dir(noop)  # нет __dict__

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 'some_attribute']

# Слайд 9. Связанные и несвязанные методы
У связанного метода первый аргумент уже зафиксирован и
равен соответствующему экземпляру:

In [7]:
class SomeClass:
    def do_something(self):
        print("Doing something.")

SomeClass().do_something # связанный

<bound method SomeClass.do_something of <__main__.SomeClass object at 0x0000000007145F48>>

In [9]:
SomeClass().do_something()

Doing something.


Несвязанному методу необходимо явно передать
экземпляр первым аргументом в момент вызова, т.к. 
это обычная функция, которая принимает один аргумент и что-то делает:

In [11]:
SomeClass.do_something

<function __main__.SomeClass.do_something(self)>

In [14]:
instance = SomeClass()

In [17]:
SomeClass.do_something(instance)

Doing something.


# Слайд 10. Свойства
Механизм свойств позволяет объявлять атрибуты,
значение которых вычисляется в момент обращения.

Можно также переопределить логику изменения и
удаления таких атрибутов.

In [38]:
from os.path import dirname

class Path:
    def __init__(self, current):
        self.current = current

    def __repr__(self):
        return "Path({})".format(self.current)

    @property
    def parent(self):
        return Path(dirname(self.current))

p = Path("./examples/some_file.txt")
p.parent

Path(./examples)

In [40]:
vars(p)  # аттрибута parent НЕТ!

{'current': './examples/some_file.txt'}

# Слайд 11. Свойства, изменение и удаление

In [41]:
class BigDataModel:
    def __init__(self):
        self._params = []
    
    @property
    def params(self):
        return self._params
    
#     Вызывается при присваивании BigDataModel().params
    @params.setter
    def params(self, new_params):
        assert all(map(lambda p: p > 0, new_params))
        self._params = new_params
    
#     Вызывается при удалении del BigDataModel().params
    @params.deleter
    def params(self):
        del self._params
    
    
model = BigDataModel()
model.params = [0.1, 0.5, 0.4]
model.params

[0.1, 0.5, 0.4]

# Слайд 12. Наследование
Синтаксис оператора **`class`** позволяет унаследовать
объявляемый класс от произвольного количества других
классов:

In [29]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial

class OtherCounter(Counter):
    def get(self):
        return self.value

Поиск имени при обращении к атрибуту или методу
ведётся сначала в `__dict__` экземпляра. Если там имя не
найдено, оно ищется в классе, а затем рекурсивно во всей
иерархии наследования

In [32]:
c = OtherCounter() # вызывает Counter.__init__
c.get() # вызывает OtherCounter.get

0

In [33]:
c.value # c.__dict__["value"]

0

# Слайд 13. Перегрузка методов и функция super

In [43]:
class Counter:
    all_counters = []

    def __init__(self, initial=0):
        self.__class__.all_counters.append(self)
        self.value = initial

class OtherCounter(Counter):
    def __init__(self, initial=0):
        self.initial = initial
        
        super().__init__(initial)

oc = OtherCounter()
vars(oc)


{'initial': 0, 'value': 0}

<font color=blue><b>Вопрос</b></font>

Как можно было бы реализовать функцию <font color=green><b>super</b></font>?

In [56]:
# супер берет 1-й аргумент (это всегода экземпляр класса)
# и вызывает __class__:
oc.__class__().__init__(4)


0

# Слайд 14. Предикат isinstance

Предикат <font color=green><b>isinstance</b></font> принимает объект и класс и
проверяет, что объект является экземпляром класса:


In [42]:
class A:
    pass

class B(A):
    pass

isinstance(B(), A)

True

В качестве второго аргумента можно также передать
кортеж классов:

In [50]:
class C:
    pass
print(isinstance(B(), (A, C)))
isinstance(B(), A) or isinstance(B(), C)

True


True

# Слайд 15. Предикат issubclass
Предикат <font color=green><b>issubclass</b></font> принимает два класса и проверяет,
что первый класс является потомком второго:

In [52]:
class A:
    pass

class B(A):
    pass

issubclass(B, A)

True

Аналогично <font color=green><b>isinstance</b></font> второй аргумент может быть
кортежем классов:

In [54]:
class C:
    pass
print(issubclass(B, (A, C)))
issubclass(B, A) or issubclass(B, C)

True


True

# Слайд 16. Множественное наследование

Python не запрещает множественное наследование, например,
можно определить следующую иерархию:

In [57]:
class A:
    def f(self):
        print("A.f")
class B:
    def f(self):
        print("B.f")
        
class C(A, B):
    pass

#  Непонятно какой метод вызовится:
C().f()  # A.f!!!

A.f


# Слайд 17.  Множественное наследование и алгоритм C3

В случае множественного наследования Python использует
`алгоритм линеаризации C3` для определения метода,
который нужно вызвать (подробности не рассматриваем, важно знать что есть специальный метод, показывающий порядок разрешения методов).
- Получить линеаризацию иерархии наследования можно с
помощью метода **mro (Method Resolution Order)**:

In [59]:
C.mro()  # == C.__mro__

[__main__.C, __main__.A, __main__.B, object]

In [64]:
# мро показывает что смотрим метод сначала в С, потом в А, потом в П
help(C.mro)

Help on built-in function mro:

mro(...) method of builtins.type instance
    mro() -> list
    return a type's method resolution order



- Результат работы алгоритма C3 далеко не всегда
тривиален, поэтому использовать сложные иерархии
множественого наследования не рекомендуется.
- Больше примеров, а также реализацию алгоритма C3 на
Python можно найти по ссылке: http://bit.ly/c3-mro

# Слайд 18. Множественное наследование и классы-примеси

- Классы-примеси позволяют выборочно модифицировать
поведение класса в предположении, что класс реализует
некоторый интерфейс.
- Продолжая пример со счётчиком:
хотим сделать счётчик для работы в многопоточной программе (не изменяя исходный класс!). Для этого создадим класс-примесь <font color=blue>ThreadSafeMixin</font>. ПОтокобезопасный счётчик
выполняет операции инкремента и чтения значения счётчика с захватом мьютекса get_lock. Класс-примесь (как бы потомок Counter) модифицирует методы Counter, оборачивая вызываемые методы родителя в мьютекс.

Т.е. предполагается что он будет потомком другого класса (примесью), содержащего методы increment и get. 
Класс-примесь нужно указывать ПЕРВЫМ в списке наследования! - (см. mro) чтобы вызываемый метод сначала вызвался у примеси, который вызовет метод базового класса Counter, выполняющего нужный функционал.

In [66]:
class ThreadSafeMixin:
    """ Класс-примесь потокобезопасного счётчика"""
    get_lock = ...
    
    def increment(self):
        with self.get_lock():
            super().increment()

    def get(self):
        with self.get_lock():
            return super().get()

# Обычный счётчик (также содержить методы инкремент и гет, если
# хотим использовать класс-примесь)
class Counter:
    """I count. That is all."""
    def __init__(self, initial=0): # конструктор
        self.value = initial       # запись атрибута

    def increment(self):
        self.value += 1

    def get(self):
        return self.value          # чтение атрибута
        
class ThreadSafeCounter(ThreadSafeMixin,
    Counter):
    pass

# Слайд 19. Декораторы классов
Синтаксис декораторов работает не только для функций,
но и для классов http://python.org/dev/peps/pep-3129:

In [30]:
@deco 
class Noop:
    pass
# == эквивалентно функции, принимающую и возвращающую класс:
class Noop:
    pass
Noop = deco(Noop)

NameError: name 'deco' is not defined

- В этом случае декоратор — это функция, которая
принимает класс и возвращает другой, возможно,
преобразованный, класс.
- Декораторы классов можно также использовать вместо
чуть более магических классов-примесей.

# Слайд 20. Словари: Декораторы классов и классы-примеси

- Декораторы классов можно использовать вместо
классов-примесей.
- Например, ThreadSafeMixin в виде декоратора класса
выглядит следующим образом образом:

In [40]:
>>> def thread_safe(cls):
    orig_increment = cls.increment
    orig_get = cls.get

    def increment(self):
        with self.get_lock():
            orig_increment(self)

    def get(self):
        with self.get_lock():
            return orig_get(self)

    cls.get_lock = ...
    cls.increment = increment
    cls.get = get
    return cls


dict_keys(['foo', 'bar'])

# Слайд 21. Декораторы классов: @singleton

In [10]:
import functools

# Здесь идея в том, что класс это функция, которую можно вызвать
# и вызывается при этом конструктор. Вот и подменим этот конструктор
# обычным способом - обёрткой, как для обычной функции!
def singleton(cls):
    instance = None
    
#     wraps работает и для класса тоже!:
    @functools.wraps(cls)
    def inner(*args, **kwargs):
        nonlocal instance
        if instance is None:
            instance = cls(*args, **kwargs)
        return instance

    return inner

@singleton
class Noop:
    "I do nothing at all."
    
# Экземпляры сигнлтона - один и тот же объект:
print(id(Noop()))
print(id(Noop()))

117121200
117121200


# Слайд 22. Декораторы классов: @deprecated


In [11]:
import warnings, functools
def deprecated(cls):
    orig_init = cls.__init__
    
    @functools.wraps(cls.__init__)
    def new_init(self, *args, **kwargs):
        warnings.warn(cls.__name__ + " is deprecated.",
                     category=DeprecationWarning)
        orig_init(self, *args, **kwargs)
    
    cls.__init__ = new_init
    return cls

@deprecated
class Counter:
    def __init__(self, initial=0):
        self.value = initial

c = Counter()

C:\Python36\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Counter is deprecated.
  


# Слайд 23. Классы: резюме
Синтаксис объявления классов в Python:
```
class SomeClass(Base1, Base2, ...):
... """Useful documentation."""
... class_attr = ...
...
... def __init__(self, some_arg):
... self.instance_attr = some_arg
...
... def do_something(self):
... pass
```
В отличие от большинства объектно-ориентированных
языков Python:
- делает передачу ссылки на экземпляр явной, self —
первый аргумент каждого метода,
- реализует механим свойств — динамически вычисляемых
атрибутов,
- поддерживает изменение классов с помощью декораторов.

# Слайд 24. “Магические” методы

- “Магическими” называются внутренние методы классов,
например, метод __init__.
- С помощью “магических” методов можно:
>- управлять доступом к атрибутам экземпляра,
>- перегрузить операторы, например, операторы сравнения
или арифметические операторы,
>- определить строковое представление экземпляра или
изменить способ его хеширования.
- Мы рассмотрим только часть наиболее используемых
методов.
- Подробное описание всех “магических” методов можно
найти в документации языка
http://bit.ly/magic-methods

# Слайд 25: “Магические” методы: __getattr__

Метод `__getattr__` вызывается при попытке прочитать
значение несуществующего атрибута. Если его НЕ определить - при попытке доступа к несуществующему объекту вызывается ошибка. 
Если же переопределим этот метод то, например, возвратим имя вызываемого аттрибута (пример безполезен):

In [12]:
class Noop:
    def __getattr__(self, name):
        return name  # identity
Noop().foobar

'foobar'

# Слайд 26. __setattr__ и __delattr__

• Методы `__setattr__` и `__delattr__` позволяют управлять
изменением значения и удалением атрибутов.

• В отличие от `__getattr__` они вызываются для всех
атрибутов, а не только для несуществующих.

• Пример, запрещающий изменять значение некоторых
атрибутов:

In [31]:
class Guarded:
    guarded = []

    def __setattr__(self, name, value):
        assert name not in self.guarded
        super().__setattr__(name, value)
        
class Noop(Guarded):
    guarded = ["foobar"]

    def __init__(self):
        self.__dict__["foobar"] = 42 # Зачем это?
        
# - здесь foobar - защищённый аттрибут, и если присваивать его значение
# через self.foobar - попадем в сетаттр и вывовется исключение. ЧЕрез
# __dict__ можно обойти обработку setattr

Полезный паттерн использования setattr и delattr - **bunch**. Bunch - класс, который оборачивает словарь и позволяет обращаться к ключам в словаре как к аттрибуту. 
Это полезно когда словарь "жирный" со строковыми ключами, например, в конфиге

In [ ]:
d = Bunch({"foo":42})  
d.attr = 42  # создаст в словаре ключ "atr" со значением 42
a.foo        # получим значение ключа "foo" = 42


# Слайд 27. Функции getattr, setattr и delattr
Функция `getattr` позволяет безопасно получить значение
атрибута экземпляра класса по его имени:

In [18]:
class Noop:
    some_attribute = 42
noop = Noop()
print(getattr(noop, "some_attribute"))
print(getattr(noop, "some_other_attribute", 100500))

42
100500


In [19]:
# Комплементарные функции setattr и delattr добавляют
# и удаляют атрибут:
setattr(noop, "some_other_attribute", 100500)
delattr(noop, "some_other_attribute")

# Слайд 28. операторы сравнения

Чтобы экземпляры класса поддерживали все операторы
сравнения, нужно реализовать внушительное количество
“магических” методов:
```
instance.__eq__(other) # instance == other
instance.__ne__(other) # instance != other
instance.__lt__(other) # instance < other
instance.__le__(other) # instance <= other
instance.__gt__(other) # instance > other
instance.__ge__(other) # instance >= other
```
В уже знакомом нам модуле functools есть декоратор,
облегчающий реализацию операторов сравнения:

In [20]:
import functools
@functools.total_ordering
class Counter:
    def __eq__(self, other):
        return self.value == other.value
    
    def __lt__(self, other): # или <=, >, >=
        return self.value < other.value

# Слайд 29. метод __call__

Метод `__call__` позволяет “вызывать” экземпляры классов,
имитируя интерфейс фнукций:

In [21]:
class Identity:
    def __call__(self, x):
        return x
Identity()(42)

42

Как это можно использовать? - в написании декораторов с ключевыми аргументами


# Слайд 30. Декораторы с аргументами на основе классов

Создадим декоратор на основе класса **trace**, который ВСЕГДА принимает аргумент `handle`. Он имеем обычный метод инит, который создаёт аттрибут `handle` со значением аргумента `handle`. Далее определяем `__call__` которая будет вызыватся для оборачиваемой функции. Здесь алгоритм обычного декоратора.
Т.е. в целом написание ключегого декоратора понятное - разделены функции инициализации и обёртки.

In [24]:
import sys

class trace:
    def __init__(self, handle):
        self.handle = handle
    
    def __call__(self, func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            print(func.__name__, args, kwargs,
                file=self.handle)
            return func(*args, **kwargs)
        return inner

@trace(sys.stderr)
def identity(x):
    return x

print(identity(42))


42


identity (42,) {}


# Слайд 31. методы для преобразования в строку
Напоминание: в Python есть две различных по смыслу
функции для преобразования объекта в строку: `repr` и `str`.

Для каждой из них существует одноимённый “магический”
метод:

In [26]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial

# по сути repr должен показывать инфу, достаточную для воспроиз-
# ведения объекта. Хорошим примером явл. отображение конструктора:
    def __repr__(self):
        return "Counter({})".format(self.value)

#     То, что приятно человеку читать
    def __str__(self):
        return "Counted to {}".format(self.value)

c = Counter(42)
c

Counter(42)

In [27]:
print(c)

Counted to 42


# Слайд 29. Расширение format через “магические” методы

In [28]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial

    def __format__(self, format_spec):
        return self.value.__format__(format_spec)

c = Counter(42)
"Counted to {:b}".format(c)

'Counted to 101010'

# Слайд 33. “Магический” метод __hash__
- Метод `__hash__` используется для вычисления значения
хеш-функции.
- Реализация по умолчанию гарантирует, что одинаковое
значение хеш функции будет только у физически
одинаковых объектов, то есть:

``x is y <=> hash(x) == hash(y).``
- Несколько очевидных рекомендаций:
- Метод `__hash__` имеет смысл реализовывать только вместе
с методом `__eq__`. При этом реализация `__hash__` должна
удовлетворять: x == y => hash(x) == hash(y)
- Для изменяемых объектов можно ограничиться только
методом `__eq__`.

# Слайд 34. “Магический” метод __bool__
- Метод `__bool__` для проверки значения на истинность,
например в условии оператора **if**.
- Для класса Counter реализация `__bool__` тривиальна:

In [29]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial

    def __bool__(self):
        return bool(self.value)

c = Counter()
if not c:
    print("No counts yet.")

No counts yet.


# Слайд 35. “Магические” методы: резюме
- “Магические” методы позволяют уточнить поведение
экземпляров класса в различных конструкциях языка.
- Например, с помощью магического метода `__str__` можно
указать способ приведения экземпляра класса, а с
помощью метода `__hash__` — алгоритм хеширования
состояния класса.
- Мы рассмотрели небольшое подмножество “магических”
методов, на самом деле их много больше: практически
любое действие с экземпляром можно специализировать
для конкретного класса